## **SIFT Featrure Extraction**

#### **Overview**

The overall size of the features for the WikiArt dataset is about `240 GB`.  All the computations were performed using several environments according to the following code (responses were not saved):

```python
import cv2
import numpy as np
from tqdm import tqdm

chunk_size = 1000
total_images = len(final_df['filename'])
sift = cv2.SIFT_create()
START = 22000
STOP = total_images

for chunk_start in tqdm(range(START, STOP, chunk_size), desc='Processing chunks'):
    chunk_end = min(chunk_start + chunk_size, total_images)
    chunk_descriptors = []
    chunk_filenames = []

    for i in range(chunk_start, chunk_end):
        path = final_df['filename'].iloc[i]
        try:
            full_path = IMG_PATH + path
            img = cv2.imread(full_path)

            if img is None:
                chunk_descriptors.append(None)
                chunk_filenames.append(path)
                continue

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, descriptors = sift.detectAndCompute(gray, None)

            if descriptors is None:
                chunk_descriptors.append(None)
            else:
                chunk_descriptors.append(descriptors)

            chunk_filenames.append(path)

        except Exception as e:
            print(f'EXCEPTION for {path}:\n{e}')
            chunk_descriptors.append(None)
            chunk_filenames.append(path)

    np.savez_compressed(f'sift_chunk_{chunk_start}_to_{chunk_end - 1}.npz',
                        descriptors_list=np.array(chunk_descriptors, dtype=object),
                        filenames=np.array(chunk_filenames))

    print(f'SAVED: {chunk_start} to {chunk_end - 1} ({len(chunk_descriptors)} images)')

    del chunk_descriptors, chunk_filenames
```

Then the obtained descriptors were processed as chunks, and then PCA and the quantization were performed:

In [ ]:
import os
import glob
import joblib
import cv2
import numpy as np
import pandas as pd
from sklearn.decomposition import IncrementalPCA
from tqdm import tqdm

To reduce the descriptor dimension (`128 -> 64`), it was decided to apply PCA. We have used Incremental PCA to train it on all the chunks with SIFT feature.

In [ ]:
NPZ_PATH = 'sift_npz/'
IPCA_OUT_DIR = 'ipca_model/'
IPCA_FILE = os.path.join(IPCA_OUT_DIR, 'ipca_final.pkl')
PCA_COMPONENTS = 64
IPCA_BATCH = 2000
SAMPLE_PER_FILE_FOR_IPCA = 1800

os.makedirs(NPZ_PATH, exist_ok=True)
os.makedirs(IPCA_OUT_DIR, exist_ok=True)

CSV_PATH = 'preprocessed files/dataset.csv'
if os.path.exists(CSV_PATH):
    df = pd.read_csv(CSV_PATH)
    test_arr = df[df['subset'] == 'test']['filename'].to_numpy()
    TEST_SET = set(os.path.basename(str(x)) for x in test_arr)
    print(f'Loaded test set from {CSV_PATH}, {len(TEST_SET)} filenames will be excluded from IPCA training.')
else:
    TEST_SET = set()
    print(f'Warning: {CSV_PATH} not found — no test exclusion will be applied.')

EXCLUDED_IMAGE_COUNT = 0
INCLUDED_IMAGE_COUNT = 0


def list_npz_files(path):
    return sorted([f for f in os.listdir(path)
                   if f.lower().endswith('.npz') and os.path.isfile(os.path.join(path, f))])


def load_npz_fullpath(fullpath):
    d = np.load(fullpath, allow_pickle=True)
    return d['descriptors_list'], d['filenames']


def read_trained_list(path):
    if not os.path.exists(path):
        return set()
    with open(path, 'r', encoding='utf8') as f:
        return set(line.strip() for line in f if line.strip())


def gather_sample_for_ipca_from_filepaths(filepaths):
    global EXCLUDED_IMAGE_COUNT, INCLUDED_IMAGE_COUNT

    for fullpath in filepaths:
        descriptors_list, filenames = load_npz_fullpath(fullpath)
        # Process each descriptor and filename
        for desc, fname in zip(descriptors_list, filenames):
            fname_base = os.path.basename(str(fname))
            if fname_base in TEST_SET:
                # Skip test images
                EXCLUDED_IMAGE_COUNT += 1
                continue
            # Include image for training
            INCLUDED_IMAGE_COUNT += 1
            if desc is None:
                continue
            desc = np.asarray(desc, dtype=np.float32)
            if desc.size == 0:
                continue
            n = len(desc)
            take = min(SAMPLE_PER_FILE_FOR_IPCA, n)
            if n > take:
                idx = np.random.choice(n, take, replace=False)
                yield desc[idx]
            else:
                yield desc


def ipca_partial_fit_on_files(filepaths, ipca, batch_size=IPCA_BATCH):
    buffer = []
    buf_count = 0
    any_sample = False
    for sample in tqdm(gather_sample_for_ipca_from_filepaths(filepaths), desc='partial_fit on files'):
        any_sample = True
        buffer.append(sample)
        buf_count += len(sample)
        # When buffer is full, train IPCA
        if buf_count >= batch_size:
            batch = np.vstack(buffer)
            ipca.partial_fit(batch)
            buffer = []
            buf_count = 0
    # Train on remaining data
    if buf_count > 0:
        batch = np.vstack(buffer)
        ipca.partial_fit(batch)
    return any_sample


def save_ipca_parts(ipca, out_dir=IPCA_OUT_DIR):
    joblib.dump(ipca, IPCA_FILE)
    np.save(os.path.join(out_dir, 'ipca_components.npy'), ipca.components_.astype(np.float32))
    if hasattr(ipca, 'mean_'):
        np.save(os.path.join(out_dir, 'ipca_mean.npy'), ipca.mean_.astype(np.float32))
    print('Saved IPCA to:', IPCA_FILE)


all_files = list_npz_files(NPZ_PATH)
if len(all_files) == 0:
    print('No .npz files found in', NPZ_PATH)
    raise SystemExit(0)

all_files_basename = [os.path.basename(p) for p in all_files]
new_files = [os.path.join(NPZ_PATH, fname) for fname in all_files_basename]

if len(new_files) == 0:
    print('No new chunks to train on.')
    raise SystemExit(0)

if os.path.exists(IPCA_FILE):
    print('Loading existing IPCA:', IPCA_FILE)
    ipca = joblib.load(IPCA_FILE)
else:
    print('No existing IPCA found. Creating new IncrementalPCA.')
    ipca = IncrementalPCA(n_components=PCA_COMPONENTS)

print('Running partial_fit on', len(new_files), 'chunks (excluding test images)...')
any_sample = ipca_partial_fit_on_files(new_files, ipca, batch_size=IPCA_BATCH)
if not any_sample:
    print('Warning: no samples were found in the provided new chunks (after excluding test images).')
else:
    save_ipca_parts(ipca, IPCA_OUT_DIR)
    append_trained_list(TRAINED_LIST, [os.path.basename(p) for p in new_files])
    print('Appended', len(new_files), 'filenames to', TRAINED_LIST)

print(f'IPCA training skipped {EXCLUDED_IMAGE_COUNT} images because they belong to the test set.')
print(f'IPCA training included {INCLUDED_IMAGE_COUNT} images from the processed chunks.')

After training the IPCA, it can be applied to the data. After the dimension reduction, we choose the top `1800` best descriptors according to their `l2` norm:

In [ ]:
NPZ_IN = 'sift_npz/'
PCA_OUT = 'sift_pca_selected/'
IPCA_FILE = 'ipca_model/ipca_final.pkl'
COMPRESSED_SIZE = 1800  # K
PCA_COMPONENTS = 64  # D2

os.makedirs(PCA_OUT, exist_ok=True)


def list_npz(path):
    return sorted([os.path.join(path, f) for f in os.listdir(path) if
                   f.endswith('.npz') and os.path.isfile(os.path.join(path, f))])


def load_chunk(path):
    d = np.load(path, allow_pickle=True)
    return d['descriptors_list'], d['filenames']


def save_pca_selected(out_path, descriptors_list, filenames):
    np.savez_compressed(out_path, descriptors_list=np.array(descriptors_list, dtype=object), filenames=filenames)


ipca = joblib.load(IPCA_FILE)
files = list_npz(NPZ_IN)

for fp in tqdm(files, desc='Processing chunks to PCA-selected'):
    outp = os.path.join(PCA_OUT, os.path.basename(fp).replace('.npz', '.pca_selected.npz'))
    if os.path.exists(outp):
        continue
    descs, filenames = load_chunk(fp)
    descs_sel = []
    for desc in descs:
        if desc is None:
            descs_sel.append(None)
            continue
        desc = np.asarray(desc, dtype=np.float32)
        if desc.size == 0:
            descs_sel.append(None)
            continue

        proj = ipca.transform(desc)  # (N, D2)
        norms = np.linalg.norm(proj, axis=1)
        k = min(COMPRESSED_SIZE, len(norms))
        idx = np.argsort(norms)[-k:]
        idx = idx[np.argsort(norms[idx])[::-1]]
        proj_sel = proj[idx].astype(np.float32)
        descs_sel.append(proj_sel)
    save_pca_selected(outp, descs_sel, filenames)

print('Done. Selected PCA files in:', PCA_OUT)

To apply the quantization, it is necessary to calculate the global minimum and maximum within all the descriptors. This was also performed chunk by chunk:

In [ ]:
PCA_SELECTED_DIR = 'sift_pca_selected/'
OUT_SCALE_FILE = 'global_pca_scale.npz'
LOW_PCT = 0.1
HIGH_PCT = 99.9
SAMPLE_PER_IMAGE = 1800
MARGIN_REL = 0.01


def list_pca_selected_files(folder):
    pattern = os.path.join(folder, '*.pca_selected.npz')
    files = sorted(glob.glob(pattern))
    return files


def sample_rows(arr, max_samples):
    n = arr.shape[0]
    if n <= max_samples:
        return arr
    idx = np.random.choice(n, max_samples, replace=False)
    return arr[idx]


def compute_global_percentile_bounds(folder,
                                     low_pct=LOW_PCT,
                                     high_pct=HIGH_PCT,
                                     sample_per_image=SAMPLE_PER_IMAGE):
    files = list_pca_selected_files(folder)
    if len(files) == 0:
        raise FileNotFoundError(f'No .pca_selected.npz files found in '{folder}'')

    per_image_lows = []
    per_image_highs = []
    images_seen = 0
    files_seen = 0

    for fp in tqdm(files, desc='Files'):
        try:
            d = np.load(fp, allow_pickle=True)
        except Exception as e:
            print(f'Warning: failed to load {fp}: {e}')
            continue
        descs = d.get('descriptors_list', None)
        if descs is None:
            continue
        files_seen += 1
        # descs is None or (k, D2) arrays
        for arr in descs:
            if arr is None:
                continue
            arr = np.asarray(arr, dtype=np.float32)
            if arr.size == 0:
                continue
            # sample rows
            try:
                sampled = sample_rows(arr, sample_per_image)
            except Exception:
                # fallback
                sampled = arr
            # compute percentiles
            # sampled shape (m, D2)
            low = np.percentile(sampled, low_pct, axis=0)
            high = np.percentile(sampled, high_pct, axis=0)
            per_image_lows.append(low)
            per_image_highs.append(high)
            images_seen += 1

    if images_seen == 0:
        raise RuntimeError('No descriptors found in any pca_selected files.')

    # stack percentiles -> (n_images, D2)
    lows_stack = np.vstack(per_image_lows)  # (N_images, D2)
    highs_stack = np.vstack(per_image_highs)  # (N_images, D2)

    # take min of lows and max of highs
    gmin = np.min(lows_stack, axis=0)
    gmax = np.max(highs_stack, axis=0)

    span = gmax - gmin
    span[span < 1e-8] = 1.0

    # apply small relative margin
    gmin = gmin - span * MARGIN_REL
    gmax = gmax + span * MARGIN_REL

    return {
        'gmin': gmin.astype(np.float32),
        'gmax': gmax.astype(np.float32),
        'files_seen': files_seen,
        'images_seen': images_seen,
        'D': gmin.shape[0]
    }


res = compute_global_percentile_bounds(
    PCA_SELECTED_DIR,
    low_pct=LOW_PCT,
    high_pct=HIGH_PCT,
    sample_per_image=SAMPLE_PER_IMAGE
)

gmin = res['gmin']
gmax = res['gmax']
np.savez_compressed(OUT_SCALE_FILE, gmin=gmin, gmax=gmax)

print('Saved global PCA scale to:', OUT_SCALE_FILE)
print('Files processed:', res['files_seen'])
print('Images considered:', res['images_seen'])

Apply the quantization and combine all the chunks:

In [ ]:
PCA_SELECTED_DIR = 'sift_pca_selected/'
SCALE_FILE = 'global_pca_scale.npz'
OUT_NPZ = 'sift_compressed_all_chunks_uint8.npz'


def list_pca_selected_files(folder):
    pattern = os.path.join(folder, '*.pca_selected.npz')
    files = sorted(glob.glob(pattern))
    return files


def load_scale(scale_path):
    d = np.load(scale_path)
    gmin = d['gmin'].astype(np.float32)
    gmax = d['gmax'].astype(np.float32)
    return gmin, gmax


def quantize_projected_array(arr_float32, gmin, gmax):
    # safety
    denom = (gmax - gmin)
    # prevent divide by zero
    denom_safe = np.where(denom == 0.0, 1.0, denom)
    normalized = (arr_float32 - gmin[np.newaxis, :]) / denom_safe[np.newaxis, :]
    normalized = np.clip(normalized, 0.0, 1.0)
    quant = np.rint(normalized * 255.0).astype(np.uint8)
    return quant


def process_all_chunks(pca_selected_dir, scale_file, out_npz_path):
    files = list_pca_selected_files(pca_selected_dir)
    if len(files) == 0:
        raise FileNotFoundError(f'No .pca_selected.npz files found in '{pca_selected_dir}'')

    gmin, gmax = load_scale(scale_file)
    D = gmin.shape[0]
    print(f'Loaded scale: PCA dim = {D}. Found {len(files)} chunk files.')

    descriptors_agg = []
    filenames_agg = []

    total_images = 0
    total_quantized_desc = 0
    skipped_empty = 0

    for fp in tqdm(files, desc='Processing chunks'):
        d = np.load(fp, allow_pickle=True)
        descs = d['descriptors_list']  # None or (k, D)
        fnames = d['filenames']  # array-like of filenames

        for arr, fname in zip(descs, fnames):
            total_images += 1
            if arr is None:
                descriptors_agg.append(None)
                filenames_agg.append(fname)
                skipped_empty += 1
                continue
            a = np.asarray(arr, dtype=np.float32)
            if a.size == 0:
                descriptors_agg.append(None)
                filenames_agg.append(fname)
                skipped_empty += 1
                continue
            if a.ndim != 2 or a.shape[1] != D:
                if a.ndim == 1 and a.size == D:
                    # single descriptor -> (1,D)
                    a = a.reshape(1, D)
                else:
                    raise RuntimeError(f'Unexpected descriptor shape {a.shape} in file {fp} for image {fname}')
            q = quantize_projected_array(a, gmin, gmax)
            descriptors_agg.append(q)
            filenames_agg.append(fname)
            total_quantized_desc += q.shape[0]

    print('Aggregated images:', total_images)
    print('Skipped (None/empty) images:', skipped_empty)
    print('Total quantized descriptors (sum of keypoints kept):', total_quantized_desc)
    # save single npz
    print('Saving final aggregated npz to:', out_npz_path)
    np.savez_compressed(out_npz_path,
                        descriptors_list=np.array(descriptors_agg, dtype=object),
                        filenames=np.array(filenames_agg, dtype=object))
    print('Saved. File size (bytes):', os.path.getsize(out_npz_path))


process_all_chunks(PCA_SELECTED_DIR, SCALE_FILE, OUT_NPZ)

Finally, the data were processed and compressed (`240 GB -> 7 GB`).